## Test №1

### 1. Create connection to db
### 2. Create connections pool to db (if needed read what is pool)

In [40]:
%%time
from sqlalchemy import create_engine, func
from sqlalchemy.orm import sessionmaker
from sqlalchemy.pool import QueuePool

from db.models import YoutubeVideo, VideoLink, Link, Product, Video, YoutubeChannel
from settings import POSTGRES_URL

from pprint import pprint

engine = create_engine(POSTGRES_URL, pool_size=20, max_overflow=20, poolclass=QueuePool)
Session = sessionmaker(bind=engine)

Wall time: 2 ms


### 3. Get random video from db and print it (will use repr method) (youtube_video)

In [5]:
%%time
with Session() as session:
    que = session.query(YoutubeVideo).filter(func.random() < 0.01).first()
    print(que)

<YoutubeVideo(('NafxNKzx0Mo', datetime.datetime(2021, 1, 20, 4, 25, 3, 869231, tzinfo=psycopg2.tz.FixedOffsetTimezone(offset=0, name=None))))>
Wall time: 621 ms


### 4. Get video by id (for example 131499061) show it

In [9]:
%%time
video_id = int(input("Print video id: "))
with Session() as session:
    que = session.query(YoutubeVideo).filter_by(id=video_id).one()
    print(que)

<YoutubeVideo(('12R39oJ_yUU', datetime.datetime(2020, 7, 24, 1, 25, 41, 356255, tzinfo=psycopg2.tz.FixedOffsetTimezone(offset=0, name=None))))>
Wall time: 642 ms


### 5. Get video by external_id (for example YhIe_koNiBQ) show it

In [11]:
%%time
video_external_id = input("Print video external id: ")
with Session() as session:
    que = session.query(YoutubeVideo).filter_by(external_id=video_external_id).one()
    print(que)

<YoutubeVideo(('YhIe_koNiBQ', datetime.datetime(2020, 7, 24, 1, 22, 9, 307694, tzinfo=psycopg2.tz.FixedOffsetTimezone(offset=0, name=None))))>
Wall time: 762 ms


### 6. Get random video with paid_promotion True

In [5]:
%%time

with Session() as session:
    a = session.query(YoutubeVideo).filter_by(paid_promotion=True).filter(func.random() < 0.01).first()
    print(a)    

<YoutubeVideo(('O-CcRx_RilU', datetime.datetime(2020, 7, 9, 21, 50, 39, 465486, tzinfo=psycopg2.tz.FixedOffsetTimezone(offset=0, name=None))))>
Wall time: 4.83 s


### 7. Get 10 videos with paid_promotion True and game_id 3

In [12]:
%%time
with Session() as session:
    que = session.query(YoutubeVideo).filter_by(paid_promotion=True, game_id=3).limit(10).all()
    pprint(que)

[<YoutubeVideo(('3nCUJZqKpu0', datetime.datetime(2020, 8, 24, 16, 58, 42, 463159, tzinfo=psycopg2.tz.FixedOffsetTimezone(offset=0, name=None))))>,
 <YoutubeVideo(('4gVpPy3lEeg', datetime.datetime(2020, 8, 23, 6, 25, 35, 124776, tzinfo=psycopg2.tz.FixedOffsetTimezone(offset=0, name=None))))>,
 <YoutubeVideo(('6u_qdVHlZxs', datetime.datetime(2020, 10, 5, 6, 44, 54, 283480, tzinfo=psycopg2.tz.FixedOffsetTimezone(offset=0, name=None))))>,
 <YoutubeVideo(('_DwHrzUv9ew', datetime.datetime(2020, 8, 24, 17, 53, 21, 862200, tzinfo=psycopg2.tz.FixedOffsetTimezone(offset=0, name=None))))>,
 <YoutubeVideo(('o4Uf1xCguTA', datetime.datetime(2020, 8, 28, 5, 7, 54, 120515, tzinfo=psycopg2.tz.FixedOffsetTimezone(offset=0, name=None))))>,
 <YoutubeVideo(('KJF3VboHGzo', datetime.datetime(2020, 10, 5, 6, 44, 54, 231147, tzinfo=psycopg2.tz.FixedOffsetTimezone(offset=0, name=None))))>,
 <YoutubeVideo(('BJ795GzBY9k', datetime.datetime(2020, 8, 24, 17, 51, 54, 763473, tzinfo=psycopg2.tz.FixedOffsetTimezone(of

### 8. How many videos are there with paid_promotion, no error existing description, name, tags and main_lang Russian?

In [8]:
%%time
with Session() as session:
    que = session.query(func.count(YoutubeVideo.id)).select_from(YoutubeVideo)
    
    que = que.filter(YoutubeVideo.paid_promotion == True, YoutubeVideo.error == None,
                     YoutubeVideo.description != None, YoutubeVideo.name != None,
                     YoutubeVideo.tags != None, YoutubeVideo.main_lang == 'ru')

    
    print(que.scalar())

26989
Wall time: 9min 19s


### 9. Show video with max duration in russian uploaded between 2020-07-23 - 2020-08-23

In [ ]:
%%time
with Session() as session:
    que = session.query(YoutubeVideo.duration_sec, func.max(YoutubeVideo.duration_sec)).\
    filter(YoutubeVideo.upload_date.between('2020-07-23', '2020-8-23'), YoutubeVideo.main_lang=='ru').first()
    print(que)

### 10. Get random video and tell if there are links?

In [9]:
%%time
with Session() as session:
    link = session.query(Video).filter(func.random() < 0.01).first().links.all()
    
    if link:
        print(f"Yes, video has a link-[count={len(count_link)}]")
    else:
        print(f"No, video no has a link")

Yes, video has a link-[count=8]
Wall time: 774 ms


### 11. Show all video links for example for video 144144693

In [30]:
%%time 
youtube_video_id = input("Print youtube video id: ")
with Session() as session:
    video_id = session.query(YoutubeVideo.external_id).filter(YoutubeVideo.id==youtube_video_id).subquery()
    links = session.query(Video).filter(Video.id.in_(video_id)).one().links.all()
    
    pprint(links)

[]
Wall time: 736 ms


### 12. Show all add video links for video flV7ylIJWp8

In [33]:
%%time
video_external_id = input("Print video external id: ")
with Session() as session:
    video = session.query(Video).filter(Video.id==video_external_id).one().links.all()

    pprint(video)

[<Link(('https://bit.ly/paella-de-pollo-y-marisco-yt', 'https://www.cocinatis.com/receta/paella-de-pollo-y-marisco?utm_source=Youtube&utm_medium=social&utm_campaign=paella-de-pollo-y-marisco', None, 'www.cocinatis.com'))>,
 <Link(('https://bit.ly/suscripcion-canal-youtube-cocinatis', 'https://www.youtube.com/user/cocinatis?sub_confirmation=1?utm_source=Youtube&utm_medium=social&utm_campaign=suscripcion-canal-youtube-cocinatis', None, 'www.youtube.com'))>,
 <Link(('https://twitter.com/cocinatis', 'https://twitter.com/cocinatis', None, 'twitter.com'))>,
 <Link(('https://www.cocinatis.com/', 'https://www.cocinatis.com/', None, 'www.cocinatis.com'))>,
 <Link(('https://www.facebook.com/cocinatis/', 'https://www.facebook.com/cocinatis/', None, 'www.facebook.com'))>,
 <Link(('https://www.instagram.com/cocinatis/', 'https://www.instagram.com/cocinatis/', None, 'www.instagram.com'))>,
 <Link(('https://www.pinterest.es/cocinatis/', 'https://www.pinterest.es/cocinatis/', None, 'www.pinterest.es')

### 13. Show all video products for video flV7ylIJWp8

In [6]:
%%time
video_external_id = input("Print video external id: ")
with Session() as session:
    subquery_links = session.query(VideoLink.columns['link_id']).filter(VideoLink.columns['video_id']==video_external_id).subquery()
    subquery_domain = session.query(Link.product_domain).filter(Link.link.in_(subquery_links)).all()
    
    pprint(subquery_domain)

[('www.cocinatis.com',),
 ('www.youtube.com',),
 ('twitter.com',),
 ('www.cocinatis.com',),
 ('www.facebook.com',),
 ('www.instagram.com',),
 ('www.pinterest.es',)]
Wall time: 581 ms


### 14. How many products have Leisure Products category?

In [35]:
%%time
product_category = "Leisure Products"
with Session() as session:
    count_product = session.query(func.count(Product.domain)).filter(Product.category==product_category).all()
    print(f"{count_product} products included in {product_category}")

[(43882,)] products included in Leisure Products
Wall time: 43.4 s


### 15. How many products have Leisure Products category for each language presented in main lang?

In [10]:
%%time
product_category = input("Print category: ")
with Session() as session:
    lang_info_cat = session.query(Product.description_lang, func.count(Product.description_lang)).\
                    filter(Product.category == product_category).group_by(Product.description_lang).all()
    
    pprint(lang_info_cat)

[('', 8053),
 ('af', 3),
 ('ar', 47),
 ('auto', 19),
 ('az', 4),
 ('be', 1),
 ('bg', 40),
 ('bn', 2),
 ('bs', 29),
 ('ca', 14),
 ('ceb', 2),
 ('co', 12),
 ('cs', 171),
 ('cy', 3),
 ('da', 50),
 ('de', 1267),
 ('el', 68),
 ('en', 24619),
 ('eo', 3),
 ('es', 1086),
 ('et', 11),
 ('eu', 7),
 ('fa', 1),
 ('fi', 81),
 ('fr', 1174),
 ('fy', 3),
 ('gd', 1),
 ('gl', 12),
 ('gu', 3),
 ('ha', 4),
 ('haw', 5),
 ('hi', 16),
 ('hr', 25),
 ('ht', 2),
 ('hu', 77),
 ('id', 79),
 ('is', 10),
 ('it', 415),
 ('iw', 33),
 ('ja', 1907),
 ('jw', 3),
 ('ka', 1),
 ('kk', 1),
 ('km', 4),
 ('kn', 1),
 ('ko', 329),
 ('ky', 2),
 ('la', 6),
 ('lb', 6),
 ('lt', 18),
 ('lv', 10),
 ('mg', 2),
 ('mi', 5),
 ('ml', 1),
 ('mr', 2),
 ('ms', 10),
 ('mt', 2),
 ('ne', 1),
 ('nl', 204),
 ('no', 46),
 ('ny', 3),
 ('pl', 479),
 ('pt', 1149),
 ('ro', 83),
 ('ru', 1181),
 ('si', 1),
 ('sk', 37),
 ('sl', 23),
 ('sm', 1),
 ('sn', 1),
 ('so', 4),
 ('sr', 6),
 ('st', 1),
 ('su', 1),
 ('sv', 99),
 ('sw', 1),
 ('te', 5),
 ('th', 90),
 

### 16. Show me all products mentioned in same videos as product betafpv.com

In [12]:
%%time
domain_product = "betafpv.com"
with Session() as session:
    subquery_link_product = session.query(Link.link).filter(Link.product_domain == domain_product).subquery()

    subquery_video_link = session.query(VideoLink.columns['video_id']).\
        filter(VideoLink.columns['link_id'].in_(subquery_link_product)).subquery()

    subquery_links = session.query(func.distinct(VideoLink.columns['link_id'])).\
        filter(VideoLink.columns['video_id'].in_(subquery_video_link)).subquery()

    subquery_domain = session.query(func.distinct(Link.product_domain)).filter(Link.link.in_(subquery_links)).all()

    pprint(subquery_domain)

[('potatojet.com',),
 ('store.steampowered.com',),
 ('iha-race.com',),
 ('www.hobbyporter.com',),
 ('halorc.uk',),
 ('www.fesliyanstudios.com',),
 ('www.hardware-programmi.com',),
 ('www.currykitten.co.uk',),
 ('www.talkable.com',),
 ('es.aliexpress.com',),
 ('stock.adobe.com',),
 ('auteldrones.com',),
 ('helination.ositracker.com',),
 ('www.airvuz.com',),
 ('www.readymaderc.com',),
 ('www.thingiverse.com',),
 ('www.liftoff-game.com',),
 ('www.xhelixfpv.com',),
 ('www.happymodel.cn',),
 ('remotepilot101.com',),
 ('fanlink.to',),
 ('www.vulcanet.shop',),
 ('vuaketqua.com',),
 ('dji-store.net',),
 ('chipolo.net',),
 ('s.click.aliexpress.com',),
 ('www.webleedfpv.com',),
 ('www.drone-fpv-racer.com',),
 ('www.epidemicsound.com',),
 ('blog.studiosport.fr',),
 ('www.appagatoconyap.it',),
 ('onlyflyingmachines.com',),
 ('triplefatgoose.com',),
 ('cdn.shopifycdn.net',),
 ('www.getfpv.com',),
 ('shareasale-analytics.com',),
 ('rc-innovations.es',),
 ('www.halfchrome.com',),
 ('www.razer.com',),

### 17. How many mentions betafpv.com product have?

In [19]:
%%time
domain_product = input("Print product domain: ")
with Session() as session:
    que = session.query(Product).filter(Product.domain==domain_product).scalar().links.count()
    
    print(que)

387
Wall time: 4.76 s


### 18. Get all products with "gnom" in translated description

In [20]:
%%time
search = f"%{'gnom'}%"
with Session() as session:
    que = session.query(Product).filter(Product.description_trans.like(search)).all()
    pprint(que)

[<Product(('you.guru', datetime.datetime(2021, 4, 14, 4, 53, 56, 382856)))>,
 <Product(('turtle-burgler.tumblr.com', datetime.datetime(2021, 4, 13, 22, 30, 19, 383865)))>,
 <Product(('9085e4a.contato.site', datetime.datetime(2021, 4, 13, 16, 17, 29, 169948)))>,
 <Product(('corvus.com.ua', datetime.datetime(2021, 4, 13, 17, 27, 29, 276204)))>,
 <Product(('xn----8sbechgrcw2ao7aj4li.xn--p1ai', datetime.datetime(2021, 4, 14, 4, 51, 26, 405377)))>,
 <Product(('zwerge24.de', datetime.datetime(2021, 4, 14, 4, 57, 44, 23795)))>,
 <Product(('www.mirenesse.com', datetime.datetime(2021, 4, 21, 3, 2, 4, 182951)))>,
 <Product(('astrosvetlanadragan.com', datetime.datetime(2021, 4, 13, 16, 42, 56, 130444)))>,
 <Product(('gnomelordgaming.tumblr.com', datetime.datetime(2021, 4, 13, 18, 33, 54, 147529)))>,
 <Product(('golden-gnomes.biz', datetime.datetime(2021, 4, 13, 18, 34, 43, 232956)))>,
 <Product(('goldenmines.biz', datetime.datetime(2021, 4, 13, 18, 34, 43, 919203)))>,
 <Product(('golden-mines.biz

### 19. Get all products mentioned in list (like ['jcaholding.com.br', 'league-legends.en.uptodown.com', 'www.ncc.se', 'kek.kek', 'mem.com'])

In [9]:
%%time
list_product = ['jcaholding.com.br', 'league-legends.en.uptodown.com', 'www.ncc.se', 'kek.kek', 'mem.com']
with Session() as session:
    que = session.query(Product).filter(Product.domain.in_(list_product)).all()
    pprint(que)

[<Product(('jcaholding.com.br', datetime.datetime(2021, 4, 19, 14, 16, 30, 611419)))>,
 <Product(('league-legends.en.uptodown.com', datetime.datetime(2021, 4, 19, 14, 16, 58, 829226)))>,
 <Product(('www.ncc.se', datetime.datetime(2021, 4, 14, 2, 32, 10, 434425)))>]
Wall time: 1.47 s


### 20. Get all channels on which products were advertised for which keywords = math

In [42]:
%%time
last_like_a = "%math%"
last_like_b = "%music%"
with Session() as session:
    
    subquery_product = session.query(Product.domain).filter(func.array_to_string(Product.keywords, ',').ilike(last_like_a))
    subquery_product = subquery_product.filter(func.array_to_string(Product.keywords, ',').ilike(last_like_b)).subquery()
    
    subquery_link = session.query(Link.link).filter(Link.product_domain.in_(subquery_product)).subquery()
    subquery_video = session.query(VideoLink.columns['video_id']).filter(VideoLink.columns['link_id'].in_(subquery_link)).subquery()
    
    subquery_youtube = session.query(YoutubeVideo.channel_id).filter(YoutubeVideo.external_id.in_(subquery_video)).subquery()
    
    channels = session.query(YoutubeChannel).filter(YoutubeChannel.id.in_(subquery_youtube)).all()
    
    pprint(channels)

[<YoutubeChannel(('UCVbL3fK5nnehn-ktyhlcRxg', datetime.datetime(2020, 2, 3, 11, 17, 29, 758758, tzinfo=psycopg2.tz.FixedOffsetTimezone(offset=0, name=None))))>,
 <YoutubeChannel(('UCnXmB2dX6od7Kvkw25g635w', datetime.datetime(2020, 7, 14, 6, 18, 41, 549859, tzinfo=psycopg2.tz.FixedOffsetTimezone(offset=0, name=None))))>,
 <YoutubeChannel(('UCYx20HUyUOijltwJIUsHHzw', datetime.datetime(2019, 3, 24, 9, 6, 46, 649072, tzinfo=psycopg2.tz.FixedOffsetTimezone(offset=0, name=None))))>,
 <YoutubeChannel(('UCK8sQmJBp8GCxrOtXWBpyEA', datetime.datetime(2020, 2, 6, 4, 56, 19, 111070, tzinfo=psycopg2.tz.FixedOffsetTimezone(offset=0, name=None))))>,
 <YoutubeChannel(('UCAlWx0jwO3JEmbBR_P3nhbA', datetime.datetime(2019, 4, 4, 7, 26, 52, 880620, tzinfo=psycopg2.tz.FixedOffsetTimezone(offset=0, name=None))))>,
 <YoutubeChannel(('UCtFp7CCkycbNtxOftDNJYHQ', datetime.datetime(2019, 4, 4, 1, 9, 52, 747708, tzinfo=psycopg2.tz.FixedOffsetTimezone(offset=0, name=None))))>,
 <YoutubeChannel(('UC-wdveKCDbbjK0_rzR2